In [1]:
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt
from PIL import Image
from scipy import signal
import cv2

import os
import random

In [2]:
path = '/home/peter/Documents/dose_estimator-git/data/data_filtered/'
path_out = '/home/peter/Documents/dose_estimator-git/data/data_def_reg'

In [3]:
elastixImageFilter = sitk.ElastixImageFilter()

AttributeError: module 'SimpleITK' has no attribute 'ElastixImageFilter'

In [ ]:


elastixImageFilter.SetFixedImage(sitk.ReadImage("fixedImage.nii"))
elastixImageFilter.SetMovingImage(sitk.ReadImage("movingImage.nii"))

parameterMapVector = sitk.VectorOfParameterMap()
parameterMapVector.append(sitk.GetDefaultParameterMap("affine"))
parameterMapVector.append(sitk.GetDefaultParameterMap("bspline"))
elastixImageFilter.SetParameterMap(parameterMapVector)

elastixImageFilter.Execute()
sitk.WriteImage(elastixImageFilter.GetResultImage())